In [ ]:
# Packages, settings & constants

# EV Routing pakcage
from ev_routing.cs_floyd_warshall import CSFloydWarshall

# Visualization packages
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import os

import numpy as np

pio.templates.default = 'plotly_white'
colors = px.colors.sequential.Jet

# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']


# Initializing Dijsktra
bl = {'lat': 52.510e0, 'lon': 13.383e0} # Bottom left corner coordinate
tr = {'lat': 52.520e0, 'lon': 13.391e0} # Top right corner coordinate

M = 30000
area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]

In [ ]:
csfw = CSFloydWarshall(area, M, n_nodes=128, n_stations=30)

latitudes = [v['lat'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] not in csfw.station_vid]
longitudes = [v['lon'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] not in csfw.station_vid]

stations_latitudes = [v['lat'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] in csfw.station_vid]
stations_longitudes = [v['lon'] for v in csfw.v.values() if v['id'] in csfw.vid[:csfw.n_nodes] and v['id'] in csfw.station_vid]

node_ids = csfw.vid[:csfw.n_nodes]
node_labels = [str(i) for i in range(len(csfw.v))]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(242, 177, 172)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.add_trace( go.Scattermapbox(
    lat = stations_latitudes,
    lon = stations_longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(0, 0, 0)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))


nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = csfw.map_center[0],
            lon = csfw.map_center[1]
        ),
        pitch = 0,
        zoom = 14,
        style = 'light'
    ),
)

nodes_map.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
nodes_map.show()

In [ ]:
csfw._stations_graph()

stations_map = go.Figure()

stations_map.add_trace( go.Scattermapbox(
    lat = stations_latitudes,
    lon = stations_longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(size = 6, color = 'rgb(0, 0, 0)', opacity=1.0),
    text = node_labels,
    hoverinfo = 'none'
))

c_id = 0
for i in range(len(csfw.stations[0])):
    for j in range(len(csfw.stations[0])):
        sids = csfw.stations[i][j]
        if not sids:
            continue
            
        if len(sids) < 3:
            continue

        print(sids)
        for sid in sids:
            uid = csfw.station_vid[sid[0]]
            vid = csfw.station_vid[sid[1]]
            lat = [csfw.v[uid]['lat'], csfw.v[vid]['lat']]
            lon = [csfw.v[uid]['lon'], csfw.v[vid]['lon']]

            stations_map.add_trace(go.Scattermapbox(
                lat = lat,
                lon = lon,
                opacity = 1,
                mode = 'lines+markers+text',
                text = [str(sid[0])],
                line = go.scattermapbox.Line(width = 5, color = colors[c_id % len(colors)]),
                marker = go.scattermapbox.Marker(size = 12, color = colors[c_id % len(colors)], opacity = 1.0, symbol = ['star', 'circle']),
            ))
        c_id += 1

# roads:
# for eid in list(csfw.e.keys())[:]:
#     e = csfw.e[eid]
#     u = csfw.v[e['u']]
#     v = csfw.v[e['v']]
    
#     lat = [u['lat'], v['lat']]
#     lon = [u['lon'], v['lon']]
    
#     stations_map.add_trace(go.Scattermapbox(
#         lat = lat,
#         lon = lon,
#         mode = 'lines+markers',
#         line = go.scattermapbox.Line(width = 3, color = colors[0],),
#         marker = go.scattermapbox.Marker(size = 8, color = colors[4], symbol=['star', 'star']),
#     ))

stations_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(lat = csfw.map_center[0], lon = csfw.map_center[1]),
        pitch = 0,
        zoom = 16,
        style = 'light'
    ),
)

stations_map.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
stations_map.show()

In [ ]:
# Benchmark
from ev_routing.cs_floyd_warshall import CSFloydWarshall

benchmark = []
for i in [2**(j+1) for j in range(8)]:
    print('Running with ' + str(i) + ' nodes!')
    benchmark.append(CSFloydWarshall(area, M, n_nodes=i, n_stations=i))

In [ ]:
colors = px.colors.sequential.Agsunset

bench_fig = make_subplots(rows=1, cols=2)

bench_FW = {
    'x': np.array([fw.n_nodes for fw in benchmark]),
    'y': np.array([fw.dt_FW for fw in benchmark]),
}
bench_FW_fit = np.polyfit(np.log10(bench_FW['x']), np.log10(bench_FW['y']), 1)
bench_FW_fit_line = bench_FW_fit[0] * np.log10(bench_FW['x']) + bench_FW_fit[1]

bench_station_graph = {
    'x': np.array([fw.n_nodes for fw in benchmark]),
    'y': np.array([fw.dt_stations_graph for fw in benchmark]),
}
bench_station_graph_fit = np.polyfit(np.log10(bench_station_graph['x']), np.log10(bench_station_graph['y']), 1)
bench_station_graph_fit_line = bench_station_graph_fit[0] * np.log10(bench_station_graph['x']) + bench_station_graph_fit[1]

bench_fig.add_trace(go.Scatter(
        x = np.log10(bench_FW['x']), y = bench_FW_fit_line,
        mode = 'lines+markers', opacity = 0.5, name = 'FW fit',
        line = dict(width = 3,color = colors[0],),
        marker = dict(size = 12,color = colors[0],),
    ), row = 1, col = 1
)

bench_fig.add_trace(go.Scatter(
        x = np.log10(bench_FW['x']), y = np.log10(bench_FW['y']),
        mode = 'lines+markers', name = 'FW',
        line = dict(width = 3,color = colors[0],),
        marker = dict(size = 12,color = colors[0],),
    ), row = 1, col = 1
)

bench_fig.add_trace(go.Scatter(
        x = np.log10(bench_station_graph['x']), y = bench_station_graph_fit_line,
        mode = 'lines+markers', opacity = 0.5, name = 'Station Graph fit',
        line = dict(width = 3,color = colors[4],),
        marker = dict(size = 12,color = colors[4],),
    ), row = 1, col = 2
)

bench_fig.add_trace(go.Scatter(
        x = np.log10(bench_station_graph['x']), y = np.log10(bench_station_graph['y']),
        mode = 'lines+markers', name = 'Station Graph',
        line = dict(width = 3,color = colors[4]),
        marker = dict(size = 12,color = colors[4])
    ), row = 1, col = 2
)

bench_fig.update_layout(height = 600, width = 800, title_text = "Benchmarking")
bench_fig.show()

print('FW O(N^' + str(bench_FW_fit[0]) + ')')
print('Station Graph O(N^' + str(bench_station_graph_fit[0]) + ')')